In [ ]:
from google.colab import drive
import os
import torch

# Mount Google Drive
drive.mount('/content/drive')

# Check available directories in Google Drive root
print("Root directories:", os.listdir('/content/drive'))
shortcut_folder_path = '/content/drive/My Drive/all_images'
files = os.listdir(shortcut_folder_path)
print("Files in all_images folder:", files)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Root directories: ['MyDrive', '.shortcut-targets-by-id', '.file-revisions-by-id', '.Trash-0', '.Encrypted']
Files in all_images folder: ['val', 'test', 'train_data.csv', 'val_data.csv', 'train']


In [ ]:
!pip install datasets

In [ ]:
!pip install open_clip_torch

In [ ]:
import open_clip
import torch
import os

# Set device for model and tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directory where processed chunks are saved in Google Drive
processed_chunks_dir = "/content/drive/MyDrive/processed_chunks"

# Directory to save model checkpoints in Google Drive
model_save_dir = "/content/drive/MyDrive/model_checkpoints"
os.makedirs(model_save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Load the OpenAI CLIP model
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Number of epochs
num_epochs = 15


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/open_clip/factory.py:372: UserWarning: These pretrained weights were trained with QuickGELU activation but the model config does not have that enabled. Consider using a model config with a "-quickgelu" suffix or enable with a flag.
  warnings.warn(


In [ ]:
import open_clip
import torch
import os
from torch.utils.data import DataLoader, TensorDataset  # Import TensorDataset here

# Set device for model and tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directory where processed chunks are saved in Google Drive
processed_chunks_dir = "/content/drive/MyDrive/processed_chunks"

# Directory to save model checkpoints in Google Drive
model_save_dir = "/content/drive/MyDrive/model_checkpoints"
os.makedirs(model_save_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Load the OpenAI CLIP model
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='openai')
model = model.to(device)

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Number of epochs
num_epochs = 15


In [ ]:
import open_clip
import torch
from torch.utils.data import DataLoader, TensorDataset
import os

# Set device for model and tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 15
max_chunks = 3  # Limit to 3 chunks

# Training loop over each epoch
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")

    # Set the model to training mode
    model.train()

    chunk_count = 0  # Counter for processed chunks

    # Loop through each chunk file in the processed chunks directory
    for chunk_file in os.listdir(processed_chunks_dir):
        if chunk_file.endswith(".pt"):
            chunk_path = os.path.join(processed_chunks_dir, chunk_file)
            chunk_data = torch.load(chunk_path, weights_only=False)  # Load the chunk file

            # Ensure `pixel_values` is a tensor, otherwise stack if it's a list
            if isinstance(chunk_data['pixel_values'], list):
                images = torch.stack([torch.tensor(img) for img in chunk_data['pixel_values']]).to(device)
            else:
                images = chunk_data['pixel_values'].to(device)

            texts = chunk_data['text']  # Changed from 'labels' to 'text'

            # Prepare inputs for CLIP
            text_inputs = open_clip.tokenize(texts).to(device)
            dataset = TensorDataset(images, text_inputs)
            dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

            # Iterate over the DataLoader for each chunk
            for images_batch, text_inputs_batch in dataloader:
                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                outputs = model(images_batch, text_inputs_batch)

                # Check the number of outputs
                print(f"Number of outputs from model: {len(outputs)}")

                # Unpacking outputs
                if len(outputs) == 3:
                    logits_per_image, logits_per_text, _ = outputs  # Ignoring the third output
                elif len(outputs) == 2:
                    logits_per_image, logits_per_text = outputs
                elif len(outputs) == 1:
                    logits_per_image = outputs[0]
                    logits_per_text = outputs[0]  # Adjust this based on the model output
                else:
                    raise ValueError(f"Unexpected number of outputs from the model: {len(outputs)}")

                targets = torch.arange(len(images_batch)).to(device)
                loss = (criterion(logits_per_image, targets) + criterion(logits_per_text, targets)) / 2

                # Backward pass and optimization step
                loss.backward()
                optimizer.step()

            # Clear chunk data from memory after processing
            del chunk_data
            torch.cuda.empty_cache()  # Clear CUDA cache to free up memory
            print(f"Completed processing chunk: {chunk_file}")

            chunk_count += 1  # Increment chunk counter
            if chunk_count >= max_chunks:  # Stop if we've processed the maximum chunks
                break

    # Save model checkpoint after each epoch
    model_save_path = os.path.join(model_save_dir, f"model_epoch_{epoch + 1}.pt")
    torch.save(model.state_dict(), model_save_path)
    print(f"Model checkpoint saved for epoch {epoch + 1} at {model_save_path}")

print("Training completed.")


Epoch 1/15
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model: 3
Number of outputs from model:

In [ ]:
import torch
import open_clip
from torch.utils.data import DataLoader, TensorDataset
import os

# Set device for evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model and move it to the device
model, _, _ = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.to(device)

# Load the model weights from epoch 3
model_checkpoint_path = os.path.join("/content/drive/My Drive/model_checkpoints", "model_epoch_3.pt")
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model.eval()  # Set the model to evaluation mode

# Prepare evaluation data (assuming processed validation/test chunks)
evaluation_chunks_dir = '/content/drive/My Drive/all_images/val'  # Directory where evaluation chunks are stored
batch_size = 8
total_correct = 0
total_samples = 0

# Loop through each chunk file in the evaluation chunks directory
for chunk_file in os.listdir(evaluation_chunks_dir):
    if chunk_file.endswith(".pt"):
        chunk_path = os.path.join(evaluation_chunks_dir, chunk_file)
        chunk_data = torch.load(chunk_path, weights_only=False)  # Load the chunk file

        # Ensure `pixel_values` is a tensor, otherwise stack if it's a list
        if isinstance(chunk_data['pixel_values'], list):
            images = torch.stack([torch.tensor(img) for img in chunk_data['pixel_values']]).to(device)
        else:
            images = chunk_data['pixel_values'].to(device)

        texts = chunk_data['text']  # Changed from 'labels' to 'text'

        # Prepare inputs for CLIP
        text_inputs = open_clip.tokenize(texts).to(device)
        dataset = TensorDataset(images, text_inputs)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        # Evaluation loop
        with torch.no_grad():  # No gradients needed for evaluation
            for images_batch, text_inputs_batch in dataloader:
                outputs = model(images_batch, text_inputs_batch)

                # Unpack outputs (adjust based on output structure)
                if len(outputs) == 3:
                    logits_per_image, logits_per_text, _ = outputs  # Ignore third output if present
                elif len(outputs) == 2:
                    logits_per_image, logits_per_text = outputs
                else:
                    logits_per_image = outputs[0]
                    logits_per_text = outputs[0]

                # Calculate accuracy (assumes label format is as per original labels)
                targets = torch.arange(len(images_batch)).to(device)
                correct_predictions = (logits_per_image.argmax(dim=-1) == targets).sum().item()
                total_correct += correct_predictions
                total_samples += images_batch.size(0)

# Check to prevent ZeroDivisionError and calculate accuracy if there are samples
if total_samples > 0:
    accuracy = total_correct / total_samples * 100
    print(f"Evaluation Accuracy: {accuracy:.2f}%")
else:
    print("No samples found for evaluation. Please check the evaluation dataset.")


<ipython-input-6-5b8aaee6a297>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


No samples found for evaluation. Please check the evaluation dataset.


In [ ]:
import open_clip
print(open_clip.list_models())


['coca_base', 'coca_roberta-ViT-B-32', 'coca_ViT-B-32', 'coca_ViT-L-14', 'convnext_base', 'convnext_base_w', 'convnext_base_w_320', 'convnext_large', 'convnext_large_d', 'convnext_large_d_320', 'convnext_small', 'convnext_tiny', 'convnext_xlarge', 'convnext_xxlarge', 'convnext_xxlarge_320', 'EVA01-g-14', 'EVA01-g-14-plus', 'EVA02-B-16', 'EVA02-E-14', 'EVA02-E-14-plus', 'EVA02-L-14', 'EVA02-L-14-336', 'MobileCLIP-B', 'MobileCLIP-S1', 'MobileCLIP-S2', 'mt5-base-ViT-B-32', 'mt5-xl-ViT-H-14', 'nllb-clip-base', 'nllb-clip-base-siglip', 'nllb-clip-large', 'nllb-clip-large-siglip', 'RN50', 'RN50-quickgelu', 'RN50x4', 'RN50x4-quickgelu', 'RN50x16', 'RN50x16-quickgelu', 'RN50x64', 'RN50x64-quickgelu', 'RN101', 'RN101-quickgelu', 'roberta-ViT-B-32', 'swin_base_patch4_window7_224', 'ViT-B-16', 'ViT-B-16-plus', 'ViT-B-16-plus-240', 'ViT-B-16-quickgelu', 'ViT-B-16-SigLIP', 'ViT-B-16-SigLIP-256', 'ViT-B-16-SigLIP-384', 'ViT-B-16-SigLIP-512', 'ViT-B-16-SigLIP-i18n-256', 'ViT-B-32', 'ViT-B-32-256', 'V

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image, UnidentifiedImageError
import open_clip
from torchvision import transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the paths
validation_images_dir = "/content/drive/My Drive/all_images/val"  # Update this path
validation_labels_dir = "/content/drive/My Drive/all_images/val"  # Update this path

# Define a transformation for the images (use the same transformation used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize based on your model's input requirements
    transforms.ToTensor(),
])

# Define a custom dataset to load images and YOLO labels
class YOLOValidationDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        # Filter to include only image files and ensure a label file exists for each image
        self.image_files = [
            f for f in sorted(os.listdir(images_dir))
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            and os.path.exists(os.path.join(labels_dir, os.path.splitext(f)[0] + ".txt"))
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.images_dir, self.image_files[idx])

        try:
            image = Image.open(image_path).convert("RGB")
        except UnidentifiedImageError:
            print(f"Warning: Unable to open image file {image_path}")
            return None  # Return None if image can't be opened

        if self.transform:
            image = self.transform(image)

        # Load label from YOLO-format file
        label_file = os.path.splitext(self.image_files[idx])[0] + ".txt"
        label_path = os.path.join(self.labels_dir, label_file)

        with open(label_path, 'r') as f:
            # Assume the YOLO label file format has the class ID as the first entry in each line
            class_id = int(f.readline().split()[0])

        return image, class_id

# Load the validation dataset
validation_dataset = YOLOValidationDataset(
    images_dir=validation_images_dir,
    labels_dir=validation_labels_dir,
    transform=transform
)

# Filter out any None results from dataset if images failed to load
filtered_data = [data for data in validation_dataset if data is not None]
if not filtered_data:
    raise ValueError("No valid data found. Ensure images and label files are properly paired.")

validation_loader = DataLoader(
    filtered_data,
    batch_size=8,
    shuffle=False
)

# The rest of the code (model loading, evaluation loop, etc.) remains the same


# The rest of the code (model loading, evaluation loop, etc.) remains the same
# Replace this part in your code accordingly

# Verify and load the model
model_name = 'ViT-B-32'  # Check with `print(open_clip.list_models())` for correct model names
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update this path

if model_name in open_clip.list_models():
    model, _, _ = open_clip.create_model_and_transforms(model_name, pretrained=False)
    model.load_state_dict(torch.load(model_checkpoint_path, map_location=device, weights_only=True))
    model = model.to(device)
    model.eval()
else:
    raise ValueError(f"Model '{model_name}' not found in open_clip. Available models: {open_clip.list_models()}")

# Define the tokenizer for text
tokenizer = open_clip.get_tokenizer(model_name)

# Evaluation loop
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, class_ids in validation_loader:
        images = images.to(device)

        # Convert class_ids to text prompts
        texts = [f"This is class {class_id}" for class_id in class_ids]
        text_inputs = tokenizer(texts).to(device)

        # Get model predictions
        outputs = model(images, text_inputs)

        # Unpack outputs
        if len(outputs) == 3:
            logits_per_image, logits_per_text, _ = outputs  # Ignoring the third output
        elif len(outputs) == 2:
            logits_per_image, logits_per_text = outputs
        else:
            logits_per_image = logits_per_text = outputs[0]

        # Calculate predictions and accuracy
        preds = logits_per_image.argmax(dim=1)
        total_correct += (preds == class_ids.to(device)).sum().item()
        total_samples += images.size(0)

# Calculate accuracy
accuracy = (total_correct / total_samples) * 100 if total_samples > 0 else 0
print(f"Evaluation Accuracy: {accuracy:.2f}%")


Evaluation Accuracy: 0.00%


In [ ]:
# Define a mapping from class_id to class_name
class_id_to_name = {
    0: 'boneanomaly',
    1: 'bonelesion',
    2: 'foreignbody',
    3: 'fracture',
    4: 'metal',
    5: 'periostealreaction',
    6: 'pronatorsign',
    7: 'softtissue',
    8: 'text'
}

# Define class text prompts
class_text_prompts = [f"This is a {class_id_to_name[i]}" for i in range(len(class_id_to_name))]


In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image, UnidentifiedImageError
from torchvision import transforms

# Define the paths
validation_images_dir = "/content/drive/My Drive/all_images/val"  # Update this path
validation_labels_dir = "/content/drive/My Drive/all_images/val"  # Update this path

# Define a transformation for the images (use the same transformation used during training)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize based on your model's input requirements
    transforms.ToTensor(),
])

# Define a custom dataset to load images and YOLO labels
class YOLOValidationDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform

        # Filter to include only image files and ensure a label file exists for each image
        self.image_files = [
            f for f in sorted(os.listdir(images_dir))
            if f.lower().endswith(('.jpg', '.jpeg', '.png'))
            and os.path.exists(os.path.join(labels_dir, os.path.splitext(f)[0] + ".txt"))
        ]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Load image
        image_path = os.path.join(self.images_dir, self.image_files[idx])

        try:
            image = Image.open(image_path).convert("RGB")
        except UnidentifiedImageError:
            print(f"Warning: Unable to open image file {image_path}")
            return None  # Return None if image can't be opened

        if self.transform:
            image = self.transform(image)

        # Load label from YOLO-format file
        label_file = os.path.splitext(self.image_files[idx])[0] + ".txt"
        label_path = os.path.join(self.labels_dir, label_file)

        try:
            with open(label_path, 'r') as f:
                # Read all lines, extract class_ids
                lines = f.readlines()
                if not lines:
                    print(f"Warning: Empty label file {label_path}")
                    return None
                class_ids = [int(line.split()[0]) for line in lines]
                # For simplicity, take the first class_id (assuming single object per image)
                class_id = class_ids[0]
        except FileNotFoundError:
            print(f"Warning: Label file {label_path} not found.")
            return None

        return image, class_id

# Load the validation dataset
validation_dataset = YOLOValidationDataset(
    images_dir=validation_images_dir,
    labels_dir=validation_labels_dir,
    transform=transform
)

# Filter out any None results from dataset if images failed to load
filtered_data = [data for data in validation_dataset if data is not None]
if not filtered_data:
    raise ValueError("No valid data found. Ensure images and label files are properly paired.")

# Create a DataLoader
validation_loader = DataLoader(
    filtered_data,
    batch_size=8,
    shuffle=False
)


In [ ]:
import open_clip

# Set device for evaluation
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model name correctly
model_name = 'ViT-B-32'  # Correct model name with hyphens

# Verify available models
available_models = open_clip.list_models()
print(f"Available models: {available_models}")

if model_name in available_models:
    model, _, preprocess = open_clip.create_model_and_transforms(model_name, pretrained=False)

    # Load the model weights from epoch 3
    model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update this path
    try:
        # Suppress the FutureWarning by using weights_only=True if loading model weights
        model.load_state_dict(torch.load(model_checkpoint_path, map_location=device), strict=False)
    except Exception as e:
        print(f"Error loading model checkpoint: {e}")
        raise

    model = model.to(device)
    model.eval()
else:
    raise ValueError(f"Model '{model_name}' not found in open_clip. Available models: {available_models}")


Available models: ['coca_base', 'coca_roberta-ViT-B-32', 'coca_ViT-B-32', 'coca_ViT-L-14', 'convnext_base', 'convnext_base_w', 'convnext_base_w_320', 'convnext_large', 'convnext_large_d', 'convnext_large_d_320', 'convnext_small', 'convnext_tiny', 'convnext_xlarge', 'convnext_xxlarge', 'convnext_xxlarge_320', 'EVA01-g-14', 'EVA01-g-14-plus', 'EVA02-B-16', 'EVA02-E-14', 'EVA02-E-14-plus', 'EVA02-L-14', 'EVA02-L-14-336', 'MobileCLIP-B', 'MobileCLIP-S1', 'MobileCLIP-S2', 'mt5-base-ViT-B-32', 'mt5-xl-ViT-H-14', 'nllb-clip-base', 'nllb-clip-base-siglip', 'nllb-clip-large', 'nllb-clip-large-siglip', 'RN50', 'RN50-quickgelu', 'RN50x4', 'RN50x4-quickgelu', 'RN50x16', 'RN50x16-quickgelu', 'RN50x64', 'RN50x64-quickgelu', 'RN101', 'RN101-quickgelu', 'roberta-ViT-B-32', 'swin_base_patch4_window7_224', 'ViT-B-16', 'ViT-B-16-plus', 'ViT-B-16-plus-240', 'ViT-B-16-quickgelu', 'ViT-B-16-SigLIP', 'ViT-B-16-SigLIP-256', 'ViT-B-16-SigLIP-384', 'ViT-B-16-SigLIP-512', 'ViT-B-16-SigLIP-i18n-256', 'ViT-B-32', 

<ipython-input-14-2250a1e08c30>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device), strict=False)

In [ ]:
# Define the tokenizer for text
try:
    tokenizer = open_clip.get_tokenizer(model_name)
except Exception as e:
    print(f"Error getting tokenizer: {e}")
    raise

# Prepare text inputs for all classes
text_inputs = tokenizer(class_text_prompts).to(device)


In [ ]:
# Evaluation loop
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in validation_loader:
        images, class_ids = batch
        images = images.to(device)

        # Get model predictions
        outputs = model(images, text_inputs)

        # Print the output structure for debugging
        if isinstance(outputs, tuple):
            print("Model returned a tuple with length:", len(outputs))
            for i, output in enumerate(outputs):
                print(f"Output[{i}]:", output)
        else:
            print("Model returned a single output:", outputs)

        # Adjust unpacking based on the actual number of elements in outputs
        if isinstance(outputs, tuple) and len(outputs) >= 2:
            logits_per_image = outputs[0]
            logits_per_text = outputs[1]
        elif isinstance(outputs, tuple) and len(outputs) == 1:
            logits_per_image = outputs[0]
            logits_per_text = None
        else:
            logits_per_image = outputs
            logits_per_text = None

        # Proceed with accuracy calculation if logits_per_image is available
        if logits_per_image is not None:
            preds = logits_per_image.argmax(dim=1)
            correct = (preds == class_ids.to(device)).sum().item()
            total_correct += correct
            total_samples += images.size(0)

            # Optional: Print a few predictions for debugging
            print(f"Predictions: {preds.cpu().numpy()}")
            print(f"Targets: {class_ids.cpu().numpy()}")
        else:
            print("Model did not return image logits; check model output format.")

# Calculate accuracy
if total_samples > 0:
    accuracy = (total_correct / total_samples) * 100
    print(f"Evaluation Accuracy: {accuracy:.2f}%")
else:
    print("No samples found for evaluation.")


Model returned a tuple with length: 3
Output[0]: tensor([[ 0.3556,  0.3485,  0.3547,  ..., -0.0055, -0.0050, -0.0060],
        [ 0.3547,  0.3491,  0.3538,  ..., -0.0054, -0.0050, -0.0060],
        [ 0.3547,  0.3494,  0.3529,  ..., -0.0054, -0.0054, -0.0059],
        ...,
        [ 0.3546,  0.3489,  0.3539,  ..., -0.0054, -0.0049, -0.0061],
        [ 0.3528,  0.3502,  0.3517,  ..., -0.0053, -0.0054, -0.0059],
        [ 0.3528,  0.3502,  0.3517,  ..., -0.0052, -0.0054, -0.0059]],
       device='cuda:0')
Output[1]: tensor([[ 3.6094e-01,  3.6379e-01,  3.5029e-01,  ...,  2.8204e-03,
         -8.4198e-03, -4.5209e-03],
        [ 3.6000e-01,  3.5930e-01,  3.4974e-01,  ...,  3.1327e-03,
         -7.7036e-03, -7.3586e-03],
        [ 3.5886e-01,  3.5900e-01,  3.4944e-01,  ...,  6.1382e-03,
         -4.7571e-03, -6.0938e-03],
        ...,
        [ 3.5941e-01,  3.6055e-01,  3.4707e-01,  ...,  4.9460e-03,
         -7.8418e-03, -7.7188e-03],
        [ 3.6067e-01,  3.6074e-01,  3.4552e-01,  ...,  5.

In [ ]:
import torch
from PIL import Image
import open_clip

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define a function to generate reports
def generate_report(image_path):
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

    # Create dummy text input as required by CLIP (adjust as needed)
    text_inputs = tokenizer(["This is a report for the image"]).to(device)

    # Get model prediction
    with torch.no_grad():
        outputs = model(image, text_inputs)

        # Inspect output structure
        print("Model output:", outputs)

        # Check if outputs is a tuple, list, or single tensor
        if isinstance(outputs, (tuple, list)) and len(outputs) >= 2:
            logits_per_image, logits_per_text = outputs[:2]
        else:
            logits_per_image = outputs  # Assume single tensor output
            logits_per_text = None

    # Convert logits to probabilities (if applicable) or generate text prediction
    if logits_per_text is not None:
        predicted_text = logits_per_text.argmax(dim=1).cpu().numpy()
    else:
        # Alternative prediction handling if only one output
        predicted_text = logits_per_image.argmax(dim=1).cpu().numpy()

    # Generate a human-readable report based on the class prediction
    report = f"Predicted report text for the image: Class {predicted_text[0]}"

    return report

# Example: Generate report for a single image
example_image_path = "/content/drive/My Drive/all_images/val/0001_1297860395_01_WRI-L1_M014.png"  # Update with actual image path
report = generate_report(example_image_path)
print("Generated Report:", report)


<ipython-input-21-7705f7e39418>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


Model output: (tensor([[ 0.3517,  0.3506,  0.3515,  0.3497,  0.3508,  0.3509,  0.3504,  0.3507,
         -0.0058, -0.0055, -0.0054, -0.0056, -0.0055, -0.0060, -0.0055, -0.0055,
         -0.0055, -0.0054, -0.0056, -0.0054, -0.0055, -0.0059, -0.0053, -0.0055,
         -0.0056, -0.0054, -0.0059, -0.0052, -0.0058, -0.0052, -0.0058, -0.0056,
         -0.0055, -0.0053, -0.0059, -0.0054, -0.0054, -0.0054, -0.0056, -0.0056,
         -0.0056, -0.0056, -0.0054, -0.0058, -0.0053, -0.0047, -0.0061, -0.0056,
         -0.0059, -0.0058, -0.0054, -0.0058, -0.0056, -0.0057, -0.0055, -0.0056,
         -0.0059, -0.0055, -0.0054, -0.0052, -0.0057, -0.0054, -0.0054, -0.0056,
         -0.0058, -0.0058, -0.0054, -0.0057, -0.0053, -0.0055, -0.0054, -0.0057,
         -0.0057, -0.0054, -0.0056, -0.0056, -0.0054, -0.0056, -0.0053, -0.0056,
         -0.0056, -0.0056, -0.0057, -0.0053, -0.0056, -0.0054, -0.0055, -0.0050,
         -0.0053, -0.0056, -0.0058, -0.0052, -0.0049, -0.0060, -0.0055, -0.0055,
         -0.0

In [ ]:
import torch
from PIL import Image
import open_clip

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B/32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define the class mappings
class_mappings = {
    0: "Bone anomaly",
    1: "Bone lesion",
    2: "Foreign body",
    3: "Fracture",
    4: "Metal",
    5: "Periosteal reaction",
    6: "Pronator sign",
    7: "Soft tissue abnormality",
    8: "Text annotation"
}

# Define a function to generate reports
def generate_report(image_path):
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

    # Create dummy text input as required by CLIP
    text_inputs = tokenizer(["This is a report for the image"]).to(device)

    # Get model prediction
    with torch.no_grad():
        outputs = model(image, text_inputs)

        # Inspect output structure
        print("Model output:", outputs)

        # Check if outputs is a tuple, list, or single tensor
        if isinstance(outputs, (tuple, list)) and len(outputs) >= 2:
            logits_per_image, logits_per_text = outputs[:2]
        else:
            logits_per_image = outputs  # Assume single tensor output
            logits_per_text = None

    # Convert logits to probabilities (if applicable) or generate text prediction
    if logits_per_text is not None:
        predicted_text = logits_per_text.argmax(dim=1).cpu().numpy()
    else:
        # Alternative prediction handling if only one output
        predicted_text = logits_per_image.argmax(dim=1).cpu().numpy()

    # Map the predicted class to its interpretation
    class_id = predicted_text[0]
    interpretation = class_mappings.get(class_id, "Unknown class")

    # Generate a human-readable report based on the class prediction
    report = f"Predicted report for the image: {interpretation} (Class ID: {class_id})"

    return report

# Example: Generate report for a single image
example_image_path = "/content/drive/My Drive/all_images/val/0001_1297860395_01_WRI-L1_M014.png"  # Update with actual image path
report = generate_report(example_image_path)
print("Generated Report:", report)


<ipython-input-22-bf64168b3ea2>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


Model output: (tensor([[ 0.3517,  0.3506,  0.3515,  0.3497,  0.3508,  0.3509,  0.3504,  0.3507,
         -0.0058, -0.0055, -0.0054, -0.0056, -0.0055, -0.0060, -0.0055, -0.0055,
         -0.0055, -0.0054, -0.0056, -0.0054, -0.0055, -0.0059, -0.0053, -0.0055,
         -0.0056, -0.0054, -0.0059, -0.0052, -0.0058, -0.0052, -0.0058, -0.0056,
         -0.0055, -0.0053, -0.0059, -0.0054, -0.0054, -0.0054, -0.0056, -0.0056,
         -0.0056, -0.0056, -0.0054, -0.0058, -0.0053, -0.0047, -0.0061, -0.0056,
         -0.0059, -0.0058, -0.0054, -0.0058, -0.0056, -0.0057, -0.0055, -0.0056,
         -0.0059, -0.0055, -0.0054, -0.0052, -0.0057, -0.0054, -0.0054, -0.0056,
         -0.0058, -0.0058, -0.0054, -0.0057, -0.0053, -0.0055, -0.0054, -0.0057,
         -0.0057, -0.0054, -0.0056, -0.0056, -0.0054, -0.0056, -0.0053, -0.0056,
         -0.0056, -0.0056, -0.0057, -0.0053, -0.0056, -0.0054, -0.0055, -0.0050,
         -0.0053, -0.0056, -0.0058, -0.0052, -0.0049, -0.0060, -0.0055, -0.0055,
         -0.0

In [ ]:
import torch
from PIL import Image
import open_clip

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define class descriptions for interpretation
class_descriptions = {
    0: "No abnormalities detected; this appears to be a normal X-ray.",
    1: "Signs of a bone fracture. Immediate medical attention is advised.",
    2: "Foreign body detected near the bone structure, which may require further investigation.",
    3: "Evidence of soft tissue swelling, possibly indicating inflammation.",
    4: "Metal implant detected in the bone structure, likely a surgical intervention.",
    5: "Pronator sign detected, suggesting possible skeletal irregularities.",
    6: "Signs of periosteal reaction, often associated with bone injury or infection.",
    7: "Possible bone lesion, which could be benign or malignant and requires further examination.",
    8: "Bone anomaly detected; additional imaging may be recommended for a thorough diagnosis."
}

# Define a function to generate reports
def generate_report(image_path):
    # Load and preprocess image
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

    # Create dummy text input as required by CLIP (adjust as needed)
    text_inputs = tokenizer(["This is a report for the image"]).to(device)

    # Get model prediction
    with torch.no_grad():
        outputs = model(image, text_inputs)

        # Inspect output structure
        if isinstance(outputs, (tuple, list)) and len(outputs) >= 2:
            logits_per_image, logits_per_text = outputs[:2]
        else:
            logits_per_image = outputs
            logits_per_text = None

    # Convert logits to class prediction
    if logits_per_text is not None:
        predicted_class = logits_per_text.argmax(dim=1).cpu().numpy()[0]
    else:
        predicted_class = logits_per_image.argmax(dim=1).cpu().numpy()[0]

    # Generate a human-readable report based on the class prediction
    report = class_descriptions.get(predicted_class, "Unknown abnormality detected.")

    return report

# Example: Generate report for a single image
example_image_path = "/content/drive/My Drive/all_images/val/0001_1297860395_01_WRI-L1_M014.png"  # Update with actual image path
report = generate_report(example_image_path)
print("Generated Report:", report)


<ipython-input-23-5d939afc106e>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


Generated Report: Signs of a bone fracture. Immediate medical attention is advised.


In [ ]:
import torch
from PIL import Image
import open_clip
import pandas as pd

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define class descriptions for interpretation
class_descriptions = {
    0: "Bone anomaly detected, indicating potential structural irregularity.",
    1: "Bone lesion detected; further evaluation may be needed to assess severity.",
    2: "Foreign body detected near bone structure, possibly requiring removal.",
    3: "Bone fracture identified; urgent medical attention is advised.",
    4: "Metal implant detected, likely a post-surgical addition.",
    5: "Periosteal reaction noted, often a sign of bone healing or infection.",
    6: "Pronator sign visible, indicating possible skeletal irregularities.",
    7: "Soft tissue swelling observed, possibly due to inflammation.",
    8: "No abnormalities detected; this appears to be a normal X-ray."
}

# Define a function to generate reports for multiple images
def generate_reports(image_paths):
    reports = []

    for image_path in image_paths:
        # Load and preprocess image
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

        # Create dummy text input as required by CLIP
        text_inputs = tokenizer(["This is a report for the image"]).to(device)

        # Get model prediction
        with torch.no_grad():
            outputs = model(image, text_inputs)
            if isinstance(outputs, (tuple, list)) and len(outputs) >= 2:
                logits_per_image, logits_per_text = outputs[:2]
            else:
                logits_per_image = outputs
                logits_per_text = None

        # Convert logits to class prediction
        if logits_per_text is not None:
            predicted_class = logits_per_text.argmax(dim=1).cpu().numpy()[0]
        else:
            predicted_class = logits_per_image.argmax(dim=1).cpu().numpy()[0]

        # Map the prediction to the class description
        report_text = class_descriptions.get(predicted_class, "Unknown abnormality detected.")

        # Store results
        reports.append({"Image Path": image_path, "Predicted Class": predicted_class, "Report": report_text})

    return pd.DataFrame(reports)

# List of 10 image paths
image_paths = [
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0007_0812696396_01_WRI-R2_M015.png",
    "/content/drive/My Drive/all_images/val/0008_1128992441_01_WRI-L2_M005.png",
    "/content/drive/My Drive/all_images/val/0091_0524008082_01_WRI-R1_F008.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png",
    "/content/drive/My Drive/all_images/val/0086_0265789777_02_WRI-L2_F008.png",
    "/content/drive/My Drive/all_images/val/0081_0559506603_01_WRI-L1_F016.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png"
]  # Update with actual paths to 10 images

# Generate reports and display in a table
reports_df = generate_reports(image_paths)
print(reports_df)


<ipython-input-24-8a267564c51c>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


                                          Image Path  Predicted Class  \
0  /content/drive/My Drive/all_images/val/0087_04...                1   
1  /content/drive/My Drive/all_images/val/0014_05...                1   
2  /content/drive/My Drive/all_images/val/0007_08...                1   
3  /content/drive/My Drive/all_images/val/0008_11...                1   
4  /content/drive/My Drive/all_images/val/0091_05...                1   
5  /content/drive/My Drive/all_images/val/0087_04...                1   
6  /content/drive/My Drive/all_images/val/0086_02...                1   
7  /content/drive/My Drive/all_images/val/0081_05...                1   
8  /content/drive/My Drive/all_images/val/0014_05...                1   
9  /content/drive/My Drive/all_images/val/0087_04...                1   

                                              Report  
0  Bone lesion detected; further evaluation may b...  
1  Bone lesion detected; further evaluation may b...  
2  Bone lesion detected; furthe

In [25]:
import torch
from PIL import Image
import open_clip
import pandas as pd

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define class descriptions for interpretation
class_descriptions = {
    0: "Bone anomaly detected, indicating potential structural irregularity.",
    1: "Bone lesion detected; further evaluation may be needed to assess severity.",
    2: "Foreign body detected near bone structure, possibly requiring removal.",
    3: "Bone fracture identified; urgent medical attention is advised.",
    4: "Metal implant detected, likely a post-surgical addition.",
    5: "Periosteal reaction noted, often a sign of bone healing or infection.",
    6: "Pronator sign visible, indicating possible skeletal irregularities.",
    7: "Soft tissue swelling observed, possibly due to inflammation.",
    8: "No abnormalities detected; this appears to be a normal X-ray."
}

# Define text descriptions for each class
text_descriptions = [
    "This X-ray shows a bone anomaly.",
    "This X-ray shows a bone lesion.",
    "This X-ray shows a foreign body.",
    "This X-ray shows a fracture.",
    "This X-ray shows a metal implant.",
    "This X-ray shows periosteal reaction.",
    "This X-ray shows a pronator sign.",
    "This X-ray shows soft tissue abnormalities.",
    "This X-ray is normal with no abnormalities."
]

# Define a function to generate reports for multiple images
def generate_reports(image_paths):
    reports = []

    for image_path in image_paths:
        # Load and preprocess image
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

        # Prepare text inputs for all classes
        text_inputs = tokenizer(text_descriptions).to(device)

        # Get model prediction
        with torch.no_grad():
            outputs = model(image, text_inputs)  # Get logits for each text input
            logits_per_image = outputs[0] if isinstance(outputs, (tuple, list)) else outputs

        # Get the index of the highest logit for class prediction
        predicted_class = logits_per_image.argmax(dim=1).cpu().numpy()[0]

        # Map the prediction to the class description
        report_text = class_descriptions.get(predicted_class, "Unknown abnormality detected.")

        # Store results
        reports.append({"Image Path": image_path, "Predicted Class": predicted_class, "Report": report_text})

    return pd.DataFrame(reports)

# List of 10 image paths
image_paths = [
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0007_0812696396_01_WRI-R2_M015.png",
    "/content/drive/My Drive/all_images/val/0008_1128992441_01_WRI-L2_M005.png",
    "/content/drive/My Drive/all_images/val/0091_0524008082_01_WRI-R1_F008.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png",
    "/content/drive/My Drive/all_images/val/0086_0265789777_02_WRI-L2_F008.png",
    "/content/drive/My Drive/all_images/val/0081_0559506603_01_WRI-L1_F016.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png"
]  # Update with actual paths to 10 images

# Generate reports and display in a table
reports_df = generate_reports(image_paths)
print(reports_df)


<ipython-input-25-02ea262f2e58>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


                                          Image Path  Predicted Class  \
0  /content/drive/My Drive/all_images/val/0087_04...                0   
1  /content/drive/My Drive/all_images/val/0014_05...                3   
2  /content/drive/My Drive/all_images/val/0007_08...                0   
3  /content/drive/My Drive/all_images/val/0008_11...                3   
4  /content/drive/My Drive/all_images/val/0091_05...                0   
5  /content/drive/My Drive/all_images/val/0087_04...                0   
6  /content/drive/My Drive/all_images/val/0086_02...                3   
7  /content/drive/My Drive/all_images/val/0081_05...                3   
8  /content/drive/My Drive/all_images/val/0014_05...                3   
9  /content/drive/My Drive/all_images/val/0087_04...                0   

                                              Report  
0  Bone anomaly detected, indicating potential st...  
1  Bone fracture identified; urgent medical atten...  
2  Bone anomaly detected, indic

In [26]:
import torch
from PIL import Image
import open_clip
import pandas as pd

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define class descriptions for interpretation
class_descriptions = {
    0: "Bone anomaly detected, indicating potential structural irregularity.",
    1: "Bone lesion detected; further evaluation may be needed to assess severity.",
    2: "Foreign body detected near bone structure, possibly requiring removal.",
    3: "Bone fracture identified; urgent medical attention is advised.",
    4: "Metal implant detected, likely a post-surgical addition.",
    5: "Periosteal reaction noted, often a sign of bone healing or infection.",
    6: "Pronator sign visible, indicating possible skeletal irregularities.",
    7: "Soft tissue swelling observed, possibly due to inflammation.",
    8: "No abnormalities detected; this appears to be a normal X-ray."
}

# Define text descriptions for each class
text_descriptions = [
    "This X-ray shows a bone anomaly.",
    "This X-ray shows a bone lesion.",
    "This X-ray shows a foreign body.",
    "This X-ray shows a fracture.",
    "This X-ray shows a metal implant.",
    "This X-ray shows periosteal reaction.",
    "This X-ray shows a pronator sign.",
    "This X-ray shows soft tissue abnormalities.",
    "This X-ray is normal with no abnormalities."
]

# Define a function to generate reports for multiple images
def generate_reports(image_paths):
    reports = []

    for image_path in image_paths:
        # Load and preprocess image
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

        # Prepare text inputs for all classes
        text_inputs = tokenizer(text_descriptions).to(device)

        # Get model prediction
        with torch.no_grad():
            outputs = model(image, text_inputs)  # Get logits for each text input
            logits_per_image = outputs[0] if isinstance(outputs, (tuple, list)) else outputs

        # Get the index of the highest logit for class prediction
        predicted_class = logits_per_image.argmax(dim=1).cpu().numpy()[0]

        # Map the prediction to the class description
        report_text = class_descriptions.get(predicted_class, "Unknown abnormality detected.")

        # Store results
        reports.append({"Image Path": image_path, "Predicted Class": predicted_class, "Report": report_text})

    return pd.DataFrame(reports)

# List of image paths
image_paths = [
    "/content/drive/My Drive/all_images/val/0086_0265789777_02_WRI-L2_F008.png",
    "/content/drive/My Drive/all_images/val/0081_0559506603_01_WRI-L1_F016.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png",
    "/content/drive/My Drive/all_images/val/0091_0524008082_01_WRI-R1_F008.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png",
    "/content/drive/My Drive/all_images/val/0086_0265789777_02_WRI-L2_F008.png",
    "/content/drive/My Drive/all_images/val/0081_0559506603_01_WRI-L1_F016.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png"
]  # Update with actual paths to the images

# Generate reports and display in a table
reports_df = generate_reports(image_paths)
print(reports_df)


<ipython-input-26-11f7b5171edb>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


                                          Image Path  Predicted Class  \
0  /content/drive/My Drive/all_images/val/0086_02...                3   
1  /content/drive/My Drive/all_images/val/0081_05...                3   
2  /content/drive/My Drive/all_images/val/0014_05...                3   
3  /content/drive/My Drive/all_images/val/0087_04...                0   
4  /content/drive/My Drive/all_images/val/0091_05...                0   
5  /content/drive/My Drive/all_images/val/0087_04...                0   
6  /content/drive/My Drive/all_images/val/0086_02...                3   
7  /content/drive/My Drive/all_images/val/0081_05...                3   
8  /content/drive/My Drive/all_images/val/0014_05...                3   
9  /content/drive/My Drive/all_images/val/0087_04...                0   

                                              Report  
0  Bone fracture identified; urgent medical atten...  
1  Bone fracture identified; urgent medical atten...  
2  Bone fracture identified; ur

In [27]:
import torch
from PIL import Image
import open_clip
import pandas as pd

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your trained model checkpoint
model_checkpoint_path = "/content/drive/My Drive/model_checkpoints/model_epoch_3.pt"  # Update with your path
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained=False)
model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))
model = model.to(device)
model.eval()

# Define tokenizer
tokenizer = open_clip.get_tokenizer('ViT-B-32')

# Define class descriptions for interpretation
class_descriptions = {
    0: "Bone anomaly detected, indicating potential structural irregularity.",
    1: "Bone lesion detected; further evaluation may be needed to assess severity.",
    2: "Foreign body detected near bone structure, possibly requiring removal.",
    3: "Bone fracture identified; urgent medical attention is advised.",
    4: "Metal implant detected, likely a post-surgical addition.",
    5: "Periosteal reaction noted, often a sign of bone healing or infection.",
    6: "Pronator sign visible, indicating possible skeletal irregularities.",
    7: "Soft tissue swelling observed, possibly due to inflammation.",
    8: "No abnormalities detected; this appears to be a normal X-ray."
}

# Define text descriptions for each class
text_descriptions = [
    "This X-ray shows a bone anomaly.",
    "This X-ray shows a bone lesion.",
    "This X-ray shows a foreign body.",
    "This X-ray shows a fracture.",
    "This X-ray shows a metal implant.",
    "This X-ray shows periosteal reaction.",
    "This X-ray shows a pronator sign.",
    "This X-ray shows soft tissue abnormalities.",
    "This X-ray is normal with no abnormalities."
]

# Define a function to generate reports for multiple images
def generate_reports(image_paths):
    reports = []

    for image_path in image_paths:
        # Load and preprocess image
        image = Image.open(image_path).convert("RGB")
        image = preprocess(image).unsqueeze(0).to(device)  # Add batch dimension

        # Prepare text inputs for all classes
        text_inputs = tokenizer(text_descriptions).to(device)

        # Get model prediction
        with torch.no_grad():
            outputs = model(image, text_inputs)  # Get logits for each text input
            logits_per_image = outputs[0] if isinstance(outputs, (tuple, list)) else outputs

        # Get the index of the highest logit for class prediction
        predicted_class = logits_per_image.argmax(dim=1).cpu().numpy()[0]

        # Map the prediction to the class description
        report_text = class_descriptions.get(predicted_class, "Unknown abnormality detected.")

        # Store results
        reports.append({
            "Image Path": image_path,
            "Predicted Class": predicted_class,
            "Report": report_text
        })

    return pd.DataFrame(reports)

# List of image paths
image_paths = [
    "/content/drive/My Drive/all_images/val/0086_0265789777_02_WRI-L2_F008.png",
    "/content/drive/My Drive/all_images/val/0081_0559506603_01_WRI-L1_F016.png",
    "/content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png",
    "/content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png"
]  # Update with actual paths to the images

# Generate reports and display in a table
reports_df = generate_reports(image_paths)

# Set display options to show full report text without truncation
pd.set_option('display.max_colwidth', None)  # Set column width to None to avoid truncation

print(reports_df)


<ipython-input-27-f69af570e58b>:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_checkpoint_path, map_location=device))


                                                                  Image Path  \
0  /content/drive/My Drive/all_images/val/0086_0265789777_02_WRI-L2_F008.png   
1  /content/drive/My Drive/all_images/val/0081_0559506603_01_WRI-L1_F016.png   
2  /content/drive/My Drive/all_images/val/0014_0516192868_01_WRI-L2_M006.png   
3  /content/drive/My Drive/all_images/val/0087_0493070649_01_WRI-R1_F003.png   

   Predicted Class  \
0                3   
1                3   
2                3   
3                0   

                                                                 Report  
0        Bone fracture identified; urgent medical attention is advised.  
1        Bone fracture identified; urgent medical attention is advised.  
2        Bone fracture identified; urgent medical attention is advised.  
3  Bone anomaly detected, indicating potential structural irregularity.  
